# Mock Classifier With Custom Endpoints Model

 * Wrap a basic python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
   * Example of using custom endpoints that are scraped by prometheus
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install seldon-core
```
 

Wrap model using s2i

## Test locally using REST

In [1]:
!make build_rest

s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.4 seldonio/mock_classifier_with_custom_endpoints_rest:1.0
---> Installing application source...
---> Installing dependencies ...
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [2]:
!docker run --name "mock_classifier_with_custom_endpoints_rest" -d --rm \
    -e PREDICTIVE_UNIT_SERVICE_PORT=5000 \
    -p 5000:5000 -p 5055:5055 seldonio/mock_classifier_with_custom_endpoints_rest:1.0

4c607cce90accb1595670f0d5353eccda6a9ab51dfc637ed7db988baac535ab0


In [3]:
#
# Call the prediction endpoint.
# Send some random features that conform to the contract
#
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[-0.886 -1.141 -0.62 ]]
RECEIVED RESPONSE:
meta {
}
data {
  names: "proba"
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.02190268365308545
        }
      }
    }
  }
}




In [4]:
#
# Call the custom endpoint.
# In this example its used for the prediction call count.
#
!curl "http://localhost:5055/prometheus_metrics"

predict_call_count 1


In [5]:
!docker rm -v "mock_classifier_with_custom_endpoints_rest" --force

mock_classifier_with_custom_endpoints_rest


## Test locally using GRPC

In [6]:
!make build_grpc

s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.4 seldonio/mock_classifier_with_custom_endpoints_grpc:1.0
---> Installing application source...
---> Installing dependencies ...
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [7]:
!docker run --name "mock_classifier_with_custom_endpoints_rest" -d --rm \
    -e PREDICTIVE_UNIT_SERVICE_PORT=5000 \
    -p 5000:5000 -p 5055:5055 seldonio/mock_classifier_with_custom_endpoints_grpc:1.0

6fc646fb348a15a063f2230b854c82b5dc4d6921135d279881e3fd86460443f8


In [8]:
#
# Call the prediction endpoint.
# Send some random features that conform to the contract,
# using NDArray.
#
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc

----------------------------------------
SENDING NEW REQUEST:

[[-0.045  0.177  0.967]]
RECEIVED RESPONSE:
meta {
}
data {
  names: "proba"
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.0724040949230513
        }
      }
    }
  }
}




In [9]:
#
# Call the prediction endpoint.
# Send some random features that conform to the contract,
# using Tensor.
#
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc --tensor

----------------------------------------
SENDING NEW REQUEST:

[[ 1.249 -0.125 -1.616]]
RECEIVED RESPONSE:
meta {
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.043911817837546704
  }
}




In [10]:
#
# Call the custom endpoint.
# In this example its used for the prediction call count.
#
!curl "http://localhost:5055/prometheus_metrics"

predict_call_count 2


In [11]:
!docker rm -v "mock_classifier_with_custom_endpoints_rest" --force

mock_classifier_with_custom_endpoints_rest


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --memory=8096 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](../../seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

In [18]:
!helm install seldon-core-analytics ../../../helm-charts/seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --set prometheus.service_type=NodePort

NAME:   seldon-core-analytics
LAST DEPLOYED: Thu May  9 16:17:01 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     0s
grafana-import-dashboards  11    0s
prometheus-rules           0     0s
prometheus-server-conf     1     0s

==> v1beta1/ClusterRoleBinding
NAME        AGE
prometheus  0s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          0s
grafana-prom-deployment  1        1        1           0          0s
prometheus-deployment    1        1        1           0          0s

==> v1/Service
NAME                      TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)       AGE
alertmanager              ClusterIP  10.99.122.106   <none>       80/TCP        0s
grafana-prom              NodePort   10.96.98.54     <none>       80:31488/TCP  0s
prometheus-node-exporter  ClusterIP  None            <

In [21]:
!eval $(minikube docker-env) && make build_rest

s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.4 seldonio/mock_classifier_with_custom_endpoints_rest:1.0
---> Installing application source...
---> Installing dependencies ...
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [22]:
#
# Create moldel deployment
#
!kubectl create -f model-deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [23]:
!kubectl rollout status deploy/test-deployment-example-2582f05

deployment "test-deployment-example-2582f05" successfully rolled out


In [33]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    seldon-deployment-example --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[-0.01   1.065 -0.108]]
RECEIVED RESPONSE:
meta {
  puid: "mr94pflheljndmt4r7n9721qbf"
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier_with_custom_endpoints_rest:1.0"
  }
}
data {
  names: "proba"
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.06907408976710562
        }
      }
    }
  }
}




In [34]:
#
# Query prometheus for the custom metrics
#
# A "custom_service" is run on port "5055" that
# is defined in model code "MeanClassifier.py".
#
# Prometeus set to scrape the model pod on port "5055"
# by the annotations in the "model-deployment.json" manifest.
#
!curl -s "$(minikube ip):$(kubectl get svc prometheus-seldon -o jsonpath='{.spec.ports[0].nodePort}')/api/v1/query?query=predict_call_count"

{"status":"success","data":{"resultType":"vector","result":[{"metric":{"__name__":"predict_call_count","app":"test-deployment-example-2582f05","instance":"172.17.0.17:5055","job":"kubernetes-pods","kubernetes_namespace":"default","kubernetes_pod_name":"test-deployment-example-2582f05-649c6c8794-xrnvq","pod_template_hash":"649c6c8794","seldon_app_classifier_1":"seldon-85d6383dfe46b983b73c236c56599001","seldon_deployment_id":"test-deployment-seldon-deployment-example"},"value":[1557418879.402,"1"]}]}}

In [30]:
#
# The prometheus metrics can also been seen in a browser
# using the url shown after running this cell.
#
# In the prometheus ui execute the expression "predict_call_count"
#
!echo "http://$(minikube ip):$(kubectl get svc prometheus-seldon -o jsonpath='{.spec.ports[0].nodePort}')"

http://192.168.39.30:30421


In [ ]:
#
# Clean up
#
!minikube delete